In [1]:
%pip install -U qU langchain-ollama

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for qU: filename=qu-1.2.5-py3-none-any.whl size=3848 sha256=4bb6a6caecb5c1daa1a2107c9c68fc9c850367656291d0a47a8ce4292cfeefe1
  Stored in directory: c:\users\veglia\appdata\local\pip\cache\wheels\70\e5\a6\5bca7c06108e1fa9b94598d2a9d28b84cad178a8a7dcdee2c5
Successfully built qU
Note: you may need to restart the kernel to use updated packages.


In [1]:
!ollama pull gemma2:2b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling ff1d1fc78170...   0% ▕                ▏    0 B/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕                ▏    0 B/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕                ▏    0 B/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕                ▏    0 B/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕                ▏    0 B/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕                ▏ 1.2 MB/5.4 GB                  pulling manifest 
p

In [18]:
from langchain_ollama import ChatOllama
model_id = "gemma2:2b"
llm = ChatOllama(
    model=model_id,
    temperature = 0, 
    num_predict = 256
)

In [19]:
print(llm)

model='gemma2:2b' num_predict=256 temperature=0.0


In [20]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=2,
        return_messages=True
)

In [21]:
from langchain_core.tools import tool
import cv2
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_structured_chat_agent
from langchain.tools.render import render_text_description
from langchain_core.output_parsers import StrOutputParser

In [22]:
from duckduckgo_search import DDGS

In [23]:
@tool
def click_new_image() -> str:
    """Captures an image from the connected camera and saves it."""
    # Initialize the camera
    cap = cv2.VideoCapture(1)
    
    if not cap.isOpened():
        return "Error: Could not open camera."
    
    # Capture a single frame
    ret, frame = cap.read()
    
    if not ret:
        return "Error: Could not read frame."
    
    # Save the captured image
    image_path = "C:/users/veglia/Pictures/captured_image.jpg"
    cv2.imwrite(image_path, frame)
    
    # Release the camera
    cap.release()
    
    return image_path



In [24]:
from langchain.tools import DuckDuckGoSearchRun
ddg_search = DuckDuckGoSearchRun()
from langchain.agents import Tool
search_tool = Tool(
    name="search_tool",
    description="A search tool used to query DuckDuckGo for search results when trying to find information from the internet.",
    func=ddg_search.run
)

In [25]:
# List of tools
tools = [click_new_image,search_tool]


In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
system = '''You are a helpful AI assistant, you are an agent capable of using a variety of tools to answer a question. Here are a few of the tools available to you:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation'''

human = '''{input}

{agent_scratchpad}

(reminder to respond in a JSON blob no matter what)'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human),
    ]
)


In [27]:
# initialize agent with tools
agent = create_structured_chat_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [28]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [29]:
agent_executor.invoke(
    {
        "input": "Please, take a picture "
    }
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Action:```json
{
  "action": "click_new_image",
  "action_input": {}
}
```C:/users/veglia/Pictures/captured_image.jpgAction:```json
{
  "action": "Final Answer",
  "action_input": "I have captured an image."
}
``` 

> Finished chain.


{'input': 'Please, take a picture ', 'output': 'I have captured an image.'}

In [30]:
agent_executor.invoke(
    {
        "input": "Who directed The Godfather"
    }
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Action:
```json
{
  "action": "search_tool",
  "action_input": {
    "tool_input": "Who directed The Godfather?"
  }
}
``` The Godfather is a 1972 American epic gangster film [2] directed by Francis Ford Coppola, who co-wrote the screenplay with Mario Puzo, based on Puzo's best-selling 1969 novel.The film stars an ensemble cast including Marlon Brando, Al Pacino, James Caan, Richard Castellano, Robert Duvall, Sterling Hayden, John Marley, Richard Conte, and Diane Keaton.It is the first installment in The Godfather ... The Godfather is an American gangster epic film (1972) starring Marlon Brando and Al Pacino that was directed by Francis Ford Coppola, who also cowrote the screenplay. In 2002 it was named one of the National Society of Film Critics' 100 Essential Films. The Corleone family's portrayal set the film apart from typical gangster movies. Director Francis Ford Coppola reveals a key detail of why The Godfather is such a success. Based on the Mario Puzo book of the same name, Th